# Dash
### Biblioteca de Python para desarrollar aplicaciones web centradas para análisis de datos

1. [Nuestra primera aplicación](#primer)
2. [Aplicación con tablas y gráficos](#table_graph)
3. [Interactividad](#inter)
4. [HTML y CSS](#HTMLCSS)
5. [Desafío](#desafio)

# 1. Nuestra primera aplicación <a class=anchor id=primer></a>

In [2]:
#Como siempre importamos lo que vamos a usar
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html, dash_table, callback, Output, Input
import pandas as pd

In [3]:
#Inicializamos la app
app = JupyterDash(__name__)
#Definimos el layout que es lo que se va a desplegar en el navegador
app.layout = html.Div(children="Hola Mundo")
#Una vez inicializada la app la corremos
app.run_server()

# 2. Aplicación con tablas y gráficos <a class=anchor id=table_graph></a>

El "Hola Mundo" es un clásico pero no termina de aprovechar todas las posibilidades de Dash

Veamos como agregar una tabla

In [4]:
#vamos a usar el mismo dataset que el de la clase de pandas
df = pd.read_csv("flights.csv")

In [29]:
#En este caso vamos a quedarnos solo con los vuelos de enero y que salieron del JFK 
df_enero_jfk = df.loc[(df.month==1) & (df.origin=="JFK")]

#Inicializamos la app
app = JupyterDash(__name__)

#Definimos el layout que es lo que se va a desplegar en el navegador
app.layout = html.Div(children=[
    html.Div(children="Aplicación con tabla"),
    #Agregamos una tabla al layout
    dash_table.DataTable(data=df_enero_jfk.to_dict("records"), page_size=10)
])
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


Ahora agreguemos un gráfico similar al de la clase de Pandas

In [36]:
import plotly.express as px #para hace gráficos con dash necesitamos usar la librería plotly
import plotly.graph_objects as go #esto nos permite mostrar gráficos de distinto tipo en una misma figura
#creamos variables para emprolijar el gráfico
margin=10
xlim = [df.dep_delay.min()-margin, df.dep_delay.max()+margin]
ylim = [df.arr_delay.min()-margin, df.arr_delay.max()+margin]
#creamos una figura para los dos tipos de gráfico 
fig_scatter=px.scatter(data_frame=df,x="dep_delay",y="arr_delay",
                range_x=xlim,range_y=ylim,
                labels={"dep_delay":"Retraso a la salida (min)","arr_delay":"Retraso a la llegada (min)"})
fig_line=px.line(x=xlim,y=ylim, color_discrete_sequence=["black"])
#unimos todo
fig_final = go.Figure(data=fig_scatter.data+fig_line.data, 
                 layout={"xaxis":fig_scatter.layout.xaxis, "yaxis":fig_scatter.layout.yaxis})

In [37]:
#Inicializamos la app
app = JupyterDash(__name__)

#Definimos el layout que es lo que se va a desplegar en el navegador
app.layout = html.Div(children=[
    html.Div(children="Aplicación con tabla y gráfico"),
    dash_table.DataTable(data=df_enero_jfk.to_dict("records"), page_size=10),
    dcc.Graph(figure=fig_final)#Agregamos el gráfico
])
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


# 3. Interactividad <a class=anchor, id=inter></a>

Bueno, muy bonita la aplicación pero todavía no podemos interactuar con ella

Primero hay que agregar un elemento con el cual el usuario pueda interactuar. En este caso es una barra desplegable o dropdown.

In [28]:
#Inicializamos la app
app = JupyterDash(__name__)

#Definimos el layout que es lo que se va a desplegar en el navegador
app.layout = html.Div(children=[
    html.Div(children="Aplicación interactiva"),
    dash_table.DataTable(data=df_enero_jfk.to_dict("records"), page_size=10),
    #agregamos el dropdown
    dcc.Dropdown(id="selector_origen",options=df.origin.unique(), multi=True,
                 placeholder="Seleccion un aeropuerto de origen"),
    dcc.Graph(id="grafico_retrasos",figure=fig_final)
])
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


Estaría bueno que al agregar o quitar elementos del despegable se modifique el gráfico. Para eso hay que agregar controles al código.

In [38]:
#Inicializamos la app
app = JupyterDash(__name__)

#Definimos el layout que es lo que se va a desplegar en el navegador
app.layout = html.Div(children=[
    html.Div(children="Aplicación interactiva"),
    #Agregamos una tabla al layout
    dash_table.DataTable(data=df_enero_jfk.to_dict("records"), page_size=10),
    dcc.Dropdown(id="selector_origen",options=df.origin.unique(),multi=True),
    dcc.Graph(id="grafico_retrasos",figure={})
])

# Agregamos los controles para construir la interacción
@callback(
    Output(component_id='grafico_retrasos', component_property='figure'),
    Input(component_id='selector_origen', component_property='value')
)
def update_graph(origenes):
    try:
        df_origenes = df.loc[df.origin.isin(origenes)]
    except:
        df_origenes = df.iloc[:0]
    fig_scatter=px.scatter(data_frame=df_origenes,x="dep_delay",y="arr_delay", color="origin",
                range_x=xlim,range_y=ylim,
                labels={"dep_delay":"Retraso a la salida (min)","arr_delay":"Retraso a la llegada (min)",
                       "origin":"Aeropuerto de origen"})
    fig_final = go.Figure(data=fig_scatter.data+fig_line.data, 
                 layout={"xaxis":fig_scatter.layout.xaxis, "yaxis":fig_scatter.layout.yaxis})
    return fig_final
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


# 4. HTML y CSS <a class=anchor id=HTMLCSS></a>

HTML permite construir la estructura de una página web mientras que CSS permite configurar el diseño.

Veamos solo HTML

In [11]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.Div("Hola Mundo"
    ),
    html.Div(children = [
        html.Div(children="¿Dónde se inventó HTML?"), 
        html.A(children="En el cern",href="https://home.cern/"),
    ]),
    html.Img(src=app.get_asset_url("/cern.png"))
])
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


Veamos CSS

In [19]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.Div(className="row",children="Hola Mundo", 
             style={'textAlign': 'center', 'color': 'red', 'fontSize': 44}
    ),
    html.Div(children = [
        html.Div(className="six columns", children="¿Dónde se inventó HTML? ", 
             style={'textAlign': 'right', 'color': 'black', 'fontSize': 22}
        ), 
        html.A(className="six columns", children="En el cern",href="https://home.cern/",
              style={'textAlign': 'left', 'fontSize': 22}
        ),
    ]),
    html.Img(src=app.get_asset_url("/cern.png"))
 ], 
 style={'textAlign': 'center'}
)
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


# 5. Desafio <a class=anchor id=desafio></a>

Ahora que vimos como hacer una aplicación de Dash que sea interactiva podemos armar una tabla que nos permita 
visualizar la cantidad de vuelos totales por aerolínea, aeropuerto de origen y mes del año. Si llegamos con el tiempo podemos hacer que sea interactiva para que nos permita elegir con que columnas agrupar.  